In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [14]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [15]:
curs.execute("select * from mimiciv_derived.gcs;")
columns_name = [desc[0] for desc in curs.description]
gcs = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [3]:
# gcs.to_csv('../outputs/gcs.csv', index=False, encoding='utf-8-sig')
gcs = pd.read_csv('../outputs/gcs.csv', parse_dates=['charttime'])

In [4]:
gcs

,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [6]:
subjectlist1 = pd.read_csv('./data/sub1list.csv', parse_dates=['admittime','dischtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [7]:
sub1_gcs = sub1list >> left_join(gcs, by=['subject_id','stay_id'])

In [8]:
sub1_gcs

,subject_id,hadm_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196.0,2131-01-11 08:00:00,15.0,1.0,0.0,1.0,1.0
1,10001884,26184834,37510196.0,2131-01-11 10:00:00,15.0,1.0,0.0,2.0,1.0
2,10001884,26184834,37510196.0,2131-01-11 12:00:00,15.0,5.0,0.0,3.0,1.0
3,10001884,26184834,37510196.0,2131-01-11 14:00:00,15.0,5.0,0.0,3.0,1.0
4,10001884,26184834,37510196.0,2131-01-11 16:00:00,15.0,5.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...
352506,19999840,21033226,38978960.0,2164-09-16 20:21:00,15.0,1.0,0.0,1.0,1.0
352507,19999840,21033226,38978960.0,2164-09-17 00:00:00,15.0,1.0,0.0,1.0,1.0
352508,19999840,21033226,38978960.0,2164-09-17 04:00:00,15.0,1.0,0.0,1.0,1.0
352509,19999840,21033226,38978960.0,2164-09-17 08:08:00,15.0,1.0,0.0,1.0,1.0


In [9]:
# null 값 처리, 중복치 확인

# null 값 확인
print(f'null count: {sub1_gcs.gcs.isna().sum()}')
sub1_gcs[sub1_gcs.gcs.isna()]   # null 값 케이스 12건 확인. charttime도 없기 때문에 제거하겠음.


null count: 12


,subject_id,hadm_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
208,10004401,29988601,NaN,NaT,NaN,NaN,NaN,NaN,NaN
17674,10496428,26731894,39824510.0,NaT,NaN,NaN,NaN,NaN,NaN
41087,11153593,26036172,34121619.0,NaT,NaN,NaN,NaN,NaN,NaN
64108,11869585,25855136,38368006.0,NaT,NaN,NaN,NaN,NaN,NaN
68341,11988196,26286088,33299263.0,NaT,NaN,NaN,NaN,NaN,NaN
158554,14593313,22338913,33451999.0,NaT,NaN,NaN,NaN,NaN,NaN
175533,15094817,24179138,30973804.0,NaT,NaN,NaN,NaN,NaN,NaN
198856,15693895,21203492,30500789.0,NaT,NaN,NaN,NaN,NaN,NaN
251225,17101881,25648226,34828491.0,NaT,NaN,NaN,NaN,NaN,NaN
286205,18098128,25408525,36740821.0,NaT,NaN,NaN,NaN,NaN,NaN


In [12]:
# null 제거
sub1_gcs_notna = sub1_gcs[sub1_gcs.gcs.notna()] 

In [13]:
# 중복치 확인 -> 없음.
## 중복치 기준: stay_id 내에서 charttime이 동일한 경우

duplicates = sub1_gcs_notna[sub1_gcs_notna.duplicated(subset=['stay_id', 'charttime'], keep=False)]
duplicates_sorted = duplicates.sort_values(by=['stay_id', 'charttime'])

duplicates_sorted

,subject_id,hadm_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable


In [15]:
# 5. hadm_id, stay_id 별로 시간순 (charttime) 정렬
sub1_gcs_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)
sub1_gcs_notna

/var/folders/1m/_xkztmr10ks17qr1v6jbfngr0000gn/T/ipykernel_6187/1654487248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub1_gcs_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)


,subject_id,hadm_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196.0,2131-01-11 08:00:00,15.0,1.0,0.0,1.0,1.0
1,10001884,26184834,37510196.0,2131-01-11 10:00:00,15.0,1.0,0.0,2.0,1.0
2,10001884,26184834,37510196.0,2131-01-11 12:00:00,15.0,5.0,0.0,3.0,1.0
3,10001884,26184834,37510196.0,2131-01-11 14:00:00,15.0,5.0,0.0,3.0,1.0
4,10001884,26184834,37510196.0,2131-01-11 16:00:00,15.0,5.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...
352506,19999840,21033226,38978960.0,2164-09-16 20:21:00,15.0,1.0,0.0,1.0,1.0
352507,19999840,21033226,38978960.0,2164-09-17 00:00:00,15.0,1.0,0.0,1.0,1.0
352508,19999840,21033226,38978960.0,2164-09-17 04:00:00,15.0,1.0,0.0,1.0,1.0
352509,19999840,21033226,38978960.0,2164-09-17 08:08:00,15.0,1.0,0.0,1.0,1.0


In [16]:
sub1_gcs_notna.to_csv('../outputs/sub1_gcs.csv', index=False, encoding='utf-8-sig')

In [17]:
sub1_gcs_notna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 352499 entries, 0 to 352510
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   subject_id  352499 non-null  int64         
 1   hadm_id     352499 non-null  int64         
 2   stay_id     352499 non-null  float64       
 3   charttime   352499 non-null  datetime64[ns]
 4   gcs         352499 non-null  float64       
 5   gcs_motor   351297 non-null  float64       
 6   gcs_verbal  351672 non-null  float64       
 7   gcs_eyes    352095 non-null  float64       
 8   gcs_unable  352499 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 26.9 MB
